


# Nvidia Question Answering Bot


[Datasets Link](https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs)

## Install necessary libraries and import it

In [1]:
!pip install datasets transformers accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have request

In [2]:
!pip install opendatasets --quiet

In [3]:
from datasets import load_dataset, Dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import re
import opendatasets as od
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load the dataset

In [4]:
od.download('https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hilmiatha
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs


100%|██████████| 400k/400k [00:00<00:00, 74.4MB/s]

Convert to pandas DataFrame

In [5]:
df = pd.read_csv('/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv')[['question', 'answer']]
df.head(5)

,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...


In [6]:
df.shape

(7108, 2)

## Cleaning the data

In [7]:
def clean_text(text:str) -> str:
  '''
  Function to clean the text by removing special characters and converting to lowercase.
  '''
  text = text.lower()
  # Replace newline characters with a space
  text = text.replace('\n', ' ')
  # Replace any character that is not a letter, digit, or space with a space
  text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
  # Replace multiple spaces with a single space
  text = re.sub(r'\s+', ' ', text).strip()
  return text

In [8]:
## Applying the cleaning function

df['question'] = df['question'].apply(clean_text)
df['answer'] = df['answer'].apply(clean_text)

In [9]:
df.head(5)

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...


## Splitting the Data

In [10]:
train = df.sample(frac=0.8, random_state=42)
test = df.drop(train.index)

In [11]:
validation = test.sample(frac=0.5, random_state=42)
test = test.drop(validation.index)

In [12]:
train

,question,answer
4982,what was the purpose of setting up the dgx rai...,in version 2 of the pipeline the dgx raid memo...
4450,when will the full programming model enhanceme...,the full programming model enhancements for th...
4513,what impact does the ptx6 memory consistency m...,the ptx6 memory consistency model introduced s...
4836,what technologies and framework did the resear...,the researchers used the matconvnet framework ...
3800,what does the source to instruction correlatio...,the source to instruction correlation view in ...
...,...,...
6665,what is the role of the head node in a cluster,the head node serves as the external interface...
6426,what does the statement without cuda it is all...,this statement emphasizes the significance of ...
5715,how can an unrolled loop improve performance f...,an unrolled loop assigns accessed array elemen...
6853,why is it important to minimize data transfers...,data transfers between the host and device are...


In [13]:
## Shapes of all the data split
print('train',train.shape)
print('validation',validation.shape)
print('test',test.shape)

train (5686, 2)
validation (711, 2)
test (711, 2)


## Model Importing

In [14]:
checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, torch_dtype=torch.float32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Tokenization (prompt engineering also)

In [15]:
def tokenize_function(examples):
    #prompt engineering
    start_prompt = 'According to the following question:\n\n'
    end_prompt = '\nAnswer:\n'
    prompt = [start_prompt + question + end_prompt for question in examples['question']]

    # tokenize the prompt
    examples['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True,return_tensors = 'pt').input_ids
    examples['labels'] = tokenizer(examples['answer'], padding='max_length', truncation=True,return_tensors = 'pt').input_ids

    return examples

## Turning into Dataset Object

In [16]:
train_data = Dataset.from_pandas(train)
train_tokenized = train_data.map(tokenize_function, batched=True)
train_tokenized = train_tokenized.remove_columns(['question','answer', '__index_level_0__' ])

Map:   0%|          | 0/5686 [00:00<?, ? examples/s]

In [17]:
train_tokenized

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 5686
})

In [18]:
validation_data = Dataset.from_pandas(validation)
validation_tokenized = validation_data.map(tokenize_function, batched=True)
validation_tokenized = validation_tokenized.remove_columns(['question','answer', '__index_level_0__' ])

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

In [19]:
test_data = Dataset.from_pandas(test)
test_tokenized = test_data.map(tokenize_function, batched=True)
test_tokenized = test_tokenized.remove_columns(['question','answer', '__index_level_0__' ])

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

## Finetuning/Model Training

In [20]:
epochs = 3
lr = 1e-3
batch_size = 2

## defining the training arguments
training_path = './training_nvidia_chatbot'
training_args = TrainingArguments(
    output_dir = training_path,
    save_total_limit = 2,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = epochs,
    learning_rate = lr,
    evaluation_strategy = 'epoch',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
## instantiate trainer object
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_tokenized,
    eval_dataset = validation_tokenized,
)

In [22]:
## train the model and save it to desired directory
trainer.train()
model_path = './nvidia_chatbot_final_model'
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Epoch,Training Loss,Validation Loss
1,0.217500,0.189960
2,0.161800,0.168190
3,0.115100,0.160835


('./nvidia_chatbot_final_model/tokenizer_config.json',
 './nvidia_chatbot_final_model/special_tokens_map.json',
 './nvidia_chatbot_final_model/spiece.model',
 './nvidia_chatbot_final_model/added_tokens.json',
 './nvidia_chatbot_final_model/tokenizer.json')

In [23]:
eval_reslts = trainer.evaluate(eval_dataset=test_tokenized)

In [24]:
eval_reslts

{'eval_loss': 0.1651710867881775,
 'eval_runtime': 79.2202,
 'eval_samples_per_second': 8.975,
 'eval_steps_per_second': 4.494,
 'epoch': 3.0}

In [88]:
q = 'what is mig?'
start_prompt = "According to the following question:\n\n"
end_prompt = '\nAnswer:\n'
full_prompt = start_prompt + q + end_prompt
print(full_prompt)

According to the following question:

what is mig?
Answer:



In [89]:
from transformers import GenerationConfig

In [90]:
checkpoint = model_path
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

In [91]:
tokenized_test = tokenizer(full_prompt, return_tensors='pt').input_ids.to(device)

In [92]:
model_output = model.generate(tokenized_test,
                              generation_config = GenerationConfig(max_new_tokens = 150))[0]

In [93]:
answer = tokenizer.decode(model_output, skip_special_tokens=True)

In [94]:
print(answer)

the migration of mig to cuda is done by migrating all previously migrated pages to mig to migrate data to gpu memory this migration ensures that gpu kernels can utilize the high memory bandwidth efficiently


In [96]:
!zip -r /content/final_model.zip /content/nvidia_chatbot_final_model

updating: content/nvidia_chatbot_final_model/ (stored 0%)
updating: content/nvidia_chatbot_final_model/spiece.model (deflated 48%)
updating: content/nvidia_chatbot_final_model/config.json (deflated 62%)
updating: content/nvidia_chatbot_final_model/tokenizer_config.json (deflated 95%)
updating: content/nvidia_chatbot_final_model/generation_config.json (deflated 29%)
updating: content/nvidia_chatbot_final_model/special_tokens_map.json (deflated 85%)
updating: content/nvidia_chatbot_final_model/tokenizer.json (deflated 74%)
updating: content/nvidia_chatbot_final_model/model.safetensors (deflated 7%)


In [98]:
!zip -r /content/training_model.zip /content/training_nvidia_chatbot

updating: content/training_nvidia_chatbot/ (stored 0%)
updating: content/training_nvidia_chatbot/runs/ (stored 0%)
updating: content/training_nvidia_chatbot/runs/Jun28_03-32-57_7c7a0318fbc4/ (stored 0%)
updating: content/training_nvidia_chatbot/runs/Jun28_03-32-57_7c7a0318fbc4/events.out.tfevents.1719553552.7c7a0318fbc4.180.1 (deflated 25%)
updating: content/training_nvidia_chatbot/runs/Jun28_03-32-57_7c7a0318fbc4/events.out.tfevents.1719545578.7c7a0318fbc4.180.0 (deflated 62%)
updating: content/training_nvidia_chatbot/checkpoint-8500/ (stored 0%)
updating: content/training_nvidia_chatbot/checkpoint-8500/optimizer.pt


zip error: Interrupted (aborting)
